# NLP

# 3. Transformers: hype

<img src="attachements/third/nlp_before.png" width="1000"/>

## План занятий

### 1. ЧБДТ: Что Было До Трансформеров
   1. При чём тут нейро-лингвистическое программирование?
   2. Почему вектора?
   3. Счётные методы: CountVectorizer, TF-IDF, добавляем контекст 
   4. Дистрибутивная семантика: Word2vec, FastText
   5. Рекуррентные нейронные сети: LSTM, GRU
### 2. Трансформеры: база
   1. Мотивация
   2. Attention из all you need?: виды attention, интуиция, реализация
   3. Архитектура Transformer: эмбеддинги, энкодер, декодер
### 3. Трансформеры: на волне хайпа
   1. BERT
   2. GPT
   3. T5

## Transformer: архитектура

Исходно Transformer - Encoder-Decoder архитектура для seq2seq задач (машинного перевода).

Задачи в NLP бывают разные, для многих задач архитектура Encoder-Decoder избыточна.

Рассмотрим три основных семейства моделей:
- Encoder (BERT-family) - NLU (natural language understanding) задачи
- Decoder (GPT-family) - NLG (natural language generation) задачи 
- Encoder-Decoder (T5 & co.) - Text2Text задачи

<img src="attachements/third/transformer-arch.png" width="1000"/>

Две стадии: pretraining + finetuning

## BERT: Bidirectional Encoder Representations from Transformers

BERT - encoder-only transformer. Основная задача: генерировать контекстуализированные эмбеддинги для слов и предложения.

Проблемы прошлых моделей:
- Омонимия - значения слов зависят от контекста
- Out-of-vocabulary слова - новые/редкие слова, опечатки
- "Статичность" эмбеддингов

<img src="attachements/third/bert_embeddings.png" width="700"/>

## Pre-training

Идея из word2vec: получить знания о структуре и смысле языка, используя огромные массивы неразмеченных данных.

### Masked language modeling

Похоже на CBOW подход из word2vec.

<img src="attachements/third/cbow.png" width="800"/>

Основные отличия - учитываем связь между словами и весь контекст предложения.

Задача: предсказать слово, которое было скрыто маской на основе всего контекста предложения.

Пайплайн:
- Выбираем 15% токенов
  - в 80% случаев заменяем на токен [MASK]
  - в 10% случаев заменяем на рандомный
  - в 10% случаев оставляем без изменений
- Предсказываем, какой токен был заменен
  - Классификация по всему словарю

<img src="attachements/third/mlm.png" width="1000"/>

### Next sentence prediction

Кроме эмбеддингов слов, мы также получить хороший эмбеддинг предложения. Можно просто усреднить (как мы делали для word2vec), но всегда ли усреднять хорошо?

Задача - предсказать, идут ли предложения друг за другом в тексте?
- Предсказание делается от специального токена [CLS]
- Предложения разделены специальным токеном [SEP]
- 50% предложения следуют друг за другом
- 50% предложения независимы (выбираем случайное из выборки)

<img src="attachements/third/nsp.png" width="1000"/>

## Fine-Tuning

Доучиваем модельку под различные задачи на небольших данных.

- Classification
  - Sentence-pair or single sentence
- Sequence labelling (NER)
- Text segmentation

<img src="attachements/third/bert-tasks.png" width="900"/>

### Токенизация

Три типа токенов:
- Позиционные эмбеддинги - для учета порядка слов
- Эмбеддинги сегментов - для разделения предложений
- Эмбеддинги токенов - привычный слой эмбеддингов

<img src="attachements/third/bert_tokenization.png" width="1000"/>

А в остальном - буквально тот же самый энкодер из Transformer'a.

### Семейка BERT

<table>
<thead>
  <tr>
    <th>Модель</th>
    <th>Саммари</th>
    <th>Описание</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>BERT</td>
    <td>База</td>
    <td>Задачи:<br>- MLM (static masking)<br>- NSP</td>
  </tr>
  <tr>
    <td>RoBERTa</td>
    <td>Улучшенный процесс обучения</td>
    <td>Особенности:<br>- Ооочень много данных (160Gb vs 16Gb)<br>- Большой размер батча (8000 vs 256)<br><br>Задачи:<br>- MLM (dynamic masking)<br>- No NSP</td>
  </tr>
  <tr>
    <td>Electra</td>
    <td>Улучшенный процесс обучения</td>
    <td>Особенности:<br>- Тренинг на всех данных, а не только маскированных<br><br>Задачи:<br>- Replaced token detection (instead of MLM)<br>- No NSP</td>
  </tr>
  <tr>
    <td>Albert</td>
    <td>Облегченный BERT</td>
    <td>Особенности:<br>- Шэринг параметров между слоями (FF / FF+MHA / MHA)<br>- Факторизация эмбеддингов<br><br>Задачи:<br>- MLM<br>- SOP (sentence order prediction)</td>
  </tr>
  <tr>
    <td>DistillBERT &amp; TinyBERT</td>
    <td>Маленькие BERT'ы, полученные за счет дистилляции</td>
    <td>Особенности:<br>- DistillBERT учится предсказывать предсказания большого BERT<br>- TinyBERT учится предсказывать в т.ч. промежуточные слои <img src="attachements/third/distill_vs_tiny.png" width="500"/> </td>
  </tr>
</tbody>
</table>

### Кодим!

## GPT: Generative pre-trained transformer

### Language modeling

Language modeling - задача предсказания следующего токена (слова / подслова / буквы) в предложении.

По сути - генерация текста.

Примеры:
- Написание контента - сказок, рассказов, рекламных текстов
- Автодополнение (кода, поискового запроса, текста)

<img src="attachements/third/gpt-2-autoregression.gif" width="1000"/>

GPT - decoder-only transformer, задача которого - авторегрессионно генерировать следущее слово.

### Хитрости и особенности

- Очень много текстов
- Очень (ОЧЕНЬ!) большие модели

<img src="attachements/third/gpt_size.png" width="800"/>

#### Эмерджентность

Большие модели с большими данными обретают новые свойства: учатся решать задачи, которым специально не учились: перевод, ответы на вопросы, NER, сегментация текста, ...

И их больше не нужно файнтюнить!

#### Как заставить модель понять, что от неё нужно?

##### Zero-shot - просто попроси

<img src="attachements/third/zero_shot.png" width="700"/>

##### One-shot - покажи пример

<img src="attachements/third/one_shot.png" width="700"/>

##### Few-shot - покажи несколько примеров

<img src="attachements/third/few_shot.png" width="700"/>

#### Качество возрастает при увеличении модели и количества примеров

<img src="attachements/third/gpt_few_shot.png" width="1000"/>

#### Качество на отдельных задачах

Модель показывает себя очень хорошо либо даже опережает специализированные SOTA решения:

- LAMBADA - нужно предсказать последнее слово в предложении. (нужно улавливать глобальный контекст)
- StoryCloze - нужно выбрать верное окончание истории из четырех вариантов.
- HellaSwag - NLI датасет; нужно выбрать логически подходящее завершение текста.

<img src="attachements/third/gpt_pqrformance.png" width="700"/>

### Prompt-Engineering

Как ещё можно добиться, чтобы модель делала то, что вы от неё хотите?

<img src="attachements/third/prompt_engineering.png" width="600"/>

Подбор "затравки" - очень мощный инструмент. Моделями можно манипулировать!
- Я дам тебе 200 долларов чаевыми, если ты мне поможешь
- Let's think step-by-step
- ...

<img src="attachements/third/prompt_tuning.jpg" width="800"/>

Сейчас - это уже целый мир, всё ограничивается только фантазией.

### Кодим!

In [107]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import json
import pandas as pd

In [108]:
from torch.utils.data import Dataset, DataLoader
import torch

In [109]:
from tqdm import tqdm

In [110]:
from pprint import pprint

In [111]:
BATCH_SIZE = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 1e-5
VERBOSE = 10

In [112]:
model = GPT2LMHeadModel.from_pretrained("ai-forever/rugpt3small_based_on_gpt2").to(DEVICE)
tokenizer = GPT2TokenizerFast.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

In [113]:
def tokenize(texts):
    return tokenizer(texts, return_attention_mask=True, return_tensors="pt", padding=True)

In [114]:
def generate(prompt, do_sample=False, **kwargs):
    tokens = tokenize(prompt)
    out_tokens = model.generate(**tokens, do_sample=do_sample, **kwargs)

    return tokenizer.batch_decode(out_tokens, ignore_special_tokens=True)[0]

In [115]:
pprint(generate("Шел медведь по лесу", max_length=128, repetition_penalty=2.5))

('Шел медведь по лесу,\n'
 '\t\t\t\xa0И вдруг увидел он: в небе над лесом — звезда. \n'
 '      И сказал ему зверь-громовержец; «Ты видишь звезду?» Он ответил так же '
 'и воскликнул с небес громоподобным голосом (в переводе на русский язык это '
 'звучит как "звезда"). Медведь поднял голову вверх к небу… А потом стал '
 'кричать во все горло... Но не мог остановиться ни один из зверей! Тогда царь '
 'приказал убить его ударом меча!.. В тот день молния ударила прямо между '
 'ними... Они упали замертво от страха перед грозой..." [1] - Ср.: http://www2')


In [116]:
with open("data/anecdotes.json") as file:
    anecdotes = json.load(file)

In [117]:
len(anecdotes)

39636

In [118]:
anecdote_tokens = tokenizer(anecdotes, max_length=2048)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [119]:
lengths = pd.Series(list(map(len, anecdote_tokens["input_ids"])))

In [120]:
lengths.describe(percentiles=[0.5, 0.95, 0.99])

count    39636.000000
mean       118.387552
std        178.214457
min          5.000000
50%         63.000000
95%        369.000000
99%        879.650000
max       2048.000000
dtype: float64

In [121]:
anecdotes = [anecdotes[i] for i in lengths[lengths < 350].index]

In [122]:
anecdotes = sorted(anecdotes, key=lambda x: len(x))

In [123]:
anecdotes[1:3]

['-Ало, это Алена?\n-Да\n-ст', 'Узбек-людоед съел пловца']

In [124]:
class AnecdoteDataset(Dataset):
    def __init__(self, anecdotes) -> None:
        self.anecdotes = anecdotes
    
    def __len__(self):
        return len(self.anecdotes)
    
    def __getitem__(self, index):
        return self.anecdotes[index]

In [125]:
def anecdote_collate(batch):
    return tokenize(batch)

In [126]:
anecdote_dataset = AnecdoteDataset(anecdotes)
anecdote_dataloader = DataLoader(anecdote_dataset, batch_size=10, shuffle=False, collate_fn=anecdote_collate)

In [127]:
next(iter(anecdote_dataloader))

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,  1140, 11512,   448, 37960,
           367, 15393,     5],
        [   17,   618,   435,    16,   481, 24818,    35,   203,    17,  4529,
           203,    17,   285],
        [    0,     0,     0,  1140,   288, 10810,    17,   623,   660,   383,
         37284, 38607,   613],
        [    0,     0,     0,    17,  1140,  1304,  8294, 23647,    35,   203,
            17,  4529,    18],
        [    0,     0,     0,     0,     0,   822, 20681,  4527,   802,   282,
          3808,   476,    18],
        [    0,     0,     0,     0,     0,     0,  6115,  1496,  7999, 40571,
           309, 32679,     8],
        [    0,     0,    17,   548,   374,    16,  9160,   356,  6036,    35,
           203,    17,   347],
        [    0,     0,     0,     0,     0,     0,     0,     0,  2997,   798,
          6249, 26133,  4038],
        [    0,     0,     0,  7148,  1594,  4961,    16,   583,   694,   279,
           578,   350,    1

Заметим:
1. Паддинг слева
2. Паддится не так много (но можно и меньше: как?)

In [128]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [133]:
def train_epoch(epoch):
    model.train()
    losses = []

    for i, input_tokens in enumerate(tqdm(anecdote_dataloader, total=len(anecdote_dataloader))):
        input_tokens = input_tokens.to(DEVICE)

        labels = input_tokens["input_ids"].clone()
        labels[labels == 0] = -100  # to ignore them when computing loss
        
        loss = model(
            **input_tokens,
            labels=labels,
            return_dict=True
        ).loss

        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        if (i + 1) % VERBOSE == 0:
            print(f"Step {i + 1}: loss - {sum(losses[-VERBOSE:-1]) / VERBOSE}")
            model.eval()
            print(generate("Шел медведь по лесу", repetition_penalty=2.5, max_length=128))
            model.train()

    epoch_loss = sum(losses) / len(losses)

    print(f"EPOCH {epoch} - epoch loss: {(epoch_loss):3f}")
    torch.cuda.empty_cache()

    return epoch_loss

In [134]:
train_epoch(0)

  0%|          | 9/3742 [00:06<38:31,  1.61it/s]

Step 10: loss - 5.33553729057312


  0%|          | 10/3742 [00:09<1:24:58,  1.37s/it]

Шел медведь по лесу,
	И в лес он не пошел. И на медведя-то никто и внимания бы ему такоого! А тут — бац… — а там уже все равно: «Батюшки!» Ну что ж? Не повезло тебе с медведем!.. Вот ты теперь кто?..» (Анекдот) 
      - Да я же тебя знаю... Ты мне как брат родной.. Я ведь тоже из Москвы приехал к вам за тобой приехать.... Так вот у меня есть друг детства -- это мой двоюродный дед Иван Иванович Шаповалов. Он живет сейчас во Франции, но мы его


  1%|          | 19/3742 [00:14<39:00,  1.59it/s]  

...

Step 60: loss - 4.4125970840454105


  2%|▏         | 60/3742 [00:55<1:43:16,  1.68s/it]

Шел медведь по лесу.
- А ты? - спросил он, когда я вышел из леса и сел на землю рядом с ним в траву.- Я!  
       И что же это за зверь такой?- спросила она меня: "А вот этот!"Я ответил:"Да".Она засмеялась:- Даааа... Это мой котенок!- Она посмотрела мне прямо глаза,- а у тебя есть кошка?!"Котёнок?"-"Нет."И тут котик сказал ей :"-Угу..."и мы пошли к ней домой...Мы её звали Клошадка....она была очень красивая!!!Попросила кота принести нам


  2%|▏         | 68/3742 [01:00<54:35,  1.12it/s]  


KeyboardInterrupt: 

## температура и разные другие параметры генерации?

## T5: Transfer learning in all its glory

А что если объединить BERT и GPT и решать оооочень много различных задач?

Получится T5 - Encoder-Decoder transformer, предназначенный для решения text2text задач.

Можно представлять различные задачи в виде Text2text:

<img src="attachements/third/t5_multitask.png" width="1000"/>

### Предобучение

Почти как BERT, только сложнее: маскируем последовательности слов.

<img src="attachements/third/t5_pretraining.png" width="1000"/>

### Обучение

Задачи:
- Перевод - переводим с одного языка на другой.
- Суммаризация - делаем краткий пересказ текста.
- Question answering - отвечаем на вопрос по тексту.
- Классификация текстов - относим текст к какой-то группе.
- Генерация текстов - сочиняем (продолжаем) текст.
- Регрессия по текстам - предсказываем число по тексту (анализ тональности, возраст, ...). 
- NLI (Natural language inference) - понимаем логическую связь предложений: продолжает, отрицает или не связано.
- Сегментация текстов - находим отрезки текста, где что-то важное нам происходит (аспектный анализ тональности, ответ на вопрос, ...)

*(Т5 училась не на все эти задачи, некоторые просто как пример.)*

<img src="attachements/third/t5_tasks.png" width="1000"/>

### Encoder-Decoder vs Decoder-only

- Но ведь GPT умеет делать то же самое!
- Да, что-то из этого может и BERT делать...

<img src="attachements/third/transformer-arch.png" width="1000"/>

#### Encoder-only

Всякие BERT, RoBERTa and co.

Любые NLU-only задачи:
- Text Classification
- Sequence labelling
- Sentiment analysis
- NLI
- QA
- Search & Recommender systems (спойлер!)

#### Decoder-only

Всякие GPT, XLNet and co.

Любые чисто генеративные задачи:
- Text completion
- Text generation
- Creative writing

И задачи, где не требуется глубокое понимание:
- диалоговые системы

#### Encoder-Decoder

Всякие Transformer, BART, T5 and co.

Задачи на понимание + генерацию:
- Machine translation
- Summarization
- QA

#### Реальность

Берем ChatGPT и погнали...

### Кодим!